In [208]:
from gurobipy import *
import pandas as pd
import numpy as np
import grblogtools as glt
from scipy.spatial.distance import squareform, pdist
import datetime
import math

In [209]:
df = pd.read_csv('data\instance01.txt', header=None)
n_S = int(df_main['n_S'])
n_C = int(df_main['n_C'])
n_L = int(df_main['n_L'])
n_K = int(df_main['n_K'])
n_D = int(df_main['n_D'])
n_T = n_D * 48
B = int(df_main['B'])//30

df_main = df.iloc[:2,:]
df_main.columns = df_main.iloc[0]
df_main = df_main[1:]

df_car =  df.iloc[3:n_C+4,:]
df_car.columns = df_car.iloc[0]
df_car = df_car.dropna(axis=1)
df_car = df_car[1:]
print(df_car)

df_car_lv = df.iloc[n_C+5:n_C+n_L+6,:]
df_car_lv.columns = df_car_lv.iloc[0]
df_car_lv = df_car_lv.dropna(axis=1)
df_car_lv = df_car_lv[1:]
print(df_car_lv)

df_order = df.iloc[n_C+n_L+7:n_C+n_L+n_K+8,:]
df_order.columns = df_order.iloc[0]
df_order = df_order.dropna(axis=1)
df_order = df_order[1:]
df_order['Time span'] = df_order.apply(lambda row: pd.to_datetime(row['Return time']) - pd.to_datetime(row['Pick-up time']), axis=1)
df_order['Time units'] = df_order['Time span'].apply(lambda x: math.ceil(x.total_seconds() / 1800))
first_period_start = pd.to_datetime('2023/01/01 00:00')
time_unit = pd.Timedelta('30 minutes')
df_order['Pick-up time (ordinal)'] = df_order.apply(lambda row: pd.to_datetime(row['Pick-up time']) - first_period_start, axis=1).apply(lambda x: math.ceil(x.total_seconds() / 1800))
df_order['Return time (ordinal)'] = df_order.apply(lambda row: pd.to_datetime(row['Return time']) - first_period_start, axis=1).apply(lambda x: math.ceil(x.total_seconds() / 1800))
df_order = pd.merge(df_order, df_car_lv, left_on='Level', right_on='Car level', how='left')
df_order = df_order.drop(['Car level'], axis=1)
print(df_order)

df_distance = df.iloc[n_C+n_L+n_K+9:n_C+n_L+n_K+n_S**2+10,:]
df_distance.columns = df_distance.iloc[0]
df_distance = df_distance.dropna(axis=1)
df_distance = df_distance[1:]
distance = df_distance.pivot(index='From', columns='To', values='Distance')
distance = distance.apply(pd.to_numeric, errors='coerce') // 30

print(distance)

3 Car ID Level Initial station
4      1     1               1
5      2     2               1
6      3     2               2
8  Car level Hour rate
9          1       200
10         2       300
  Order ID Level Pick-up station Return station      Pick-up time  \
0        1     1               2              1  2023/01/02 09:00   
1        2     1               1              1  2023/01/01 00:30   
2        3     1               2              1  2023/01/02 18:00   
3        4     2               1              2  2023/01/01 08:30   
4        5     2               2              1  2023/01/01 09:30   

        Return time       Time span  Time units  Pick-up time (ordinal)  \
0  2023/01/03 12:00 1 days 03:00:00          54                      66   
1  2023/01/02 10:30 1 days 10:00:00          68                       1   
2  2023/01/03 02:00 0 days 08:00:00          16                      84   
3  2023/01/02 14:30 1 days 06:00:00          60                      17   
4  2023/01/02 02:

In [210]:
eg1 = Model("eg1")

#-------- Add variables as a list ---------#
# wimt = 1 if car m is at station i in period t; wimt = 0 o.w.
w = []
for i in range(n_S):
    w.append([])
    for m in range(n_C):
        w[i].append([])
        for t in range(n_T):
            w[i][m].append(eg1.addVar(lb = 0, vtype = GRB.BINARY, name = "w" + str(i+1) + str(m+1) + str(t+1)))

# zpqm = 1 if order p and order q are on car m and order p is before order q
z = []
for p in range(n_K):
    z.append([])
    for q in range(n_K):
        z[p].append([])
        for m in range(n_C):
            z[p][q].append(eg1.addVar(lb = 0, vtype = GRB.BINARY, name = "z" + str(p+1) + str(q+1) + str(m+1)))

# xmk = 1 if order k is assigned to car m
x = []
for m in range(n_C):
    x.append([])
    for k in range(n_K):
        x[m].append(eg1.addVar(lb = 0, vtype = GRB.BINARY, name = "x" + str(m+1) + str(k+1)))

# Rk = revenue of order k
R = []
for k in range(n_K):
    time_units = int(df_order.loc[k, 'Time units'])
    hour_rate = int(df_order.loc[k, 'Hour rate'])
    R.append(eg1.addVar(lb=time_units*hour_rate, vtype=GRB.INTEGER, name="R" + str(k+1)))

# rk = rental time for order k
r = []
for k in range(n_K):
    time_units = int(df_order.loc[k, 'Time units'])
    r.append(eg1.addVar(lb=time_units, vtype=GRB.INTEGER, name="r" + str(k+1)))

# lv_Dk = demanded car level of order k
lv_Dk = []
for k in range(n_K):
    level_demanded = int(df_order.loc[k, 'Level'])
    lv_Dk.append(eg1.addVar(lb=level_demanded, vtype=GRB.INTEGER, name="lv_Dk" + str(k+1)))

# # lv_Sm = supplied car level of car m
# lv_Sm = []
# for m in range(n_C):
#     level_supplied = int(df_car.loc[m, 'Level'])
#     lv_Sm.append(eg1.addVar(lb=level_supplied, vtype=GRB.INTEGER, name="lv_Sm" + str(m+1)))

# rtij = rearrangment time to move car from station i to station j
rt = []
for i in range(n_S):
    rt.append([])
    for j in range(n_S):
        rt[i].append(eg1.addVar(lb=distance.iloc[j, i], vtype=GRB.INTEGER, name="rt" + str(i+1) + str(j+1)))

# Ck = completion time of order k
C = []
for k in range(n_K):
    C.append(eg1.addVar(lb=0, vtype = GRB.INTEGER, name = "C" + str(k+1)))

# Om = latest completion time of orders on car m
O = []
for m in range(n_C):
    O.append(eg1.addVar(lb=0, vtype = GRB.INTEGER, name = "O" + str(m+1)))

Cmax = eg1.addVar(lb = 48*n_D, vtype = GRB.INTEGER, name = "Cmax")
A = eg1.addVar(lb = 10000, vtype = GRB.INTEGER, name = "a large number")

In [211]:
eg1.setObjective(
    quicksum(R[k] for k in range(n_K))\
    ,GRB.MAXIMIZE
)

In [212]:
eg1.addConstrs((quicksum(x[m][k] for m in range(n_C)) <= 1 for k in range(n_K)), "每份order對應到一台車(但也可以不接單)")
eg1.addConstrs((O[m] <= Cmax for m in range(n_C)), "links the individual car makespan to the overall schedule makespan")
eg1.addConstrs((x[m][p]+x[m][q] <= z[p][q][m]+z[q][p][m]+1) for m in range(n_C) for p in range(n_K) for q in range(n_K))
eg1.addConstrs((C[p]+r[p]-C[q] <=  A * ( 1- z[p][q][m])) for m in range(n_C) for p in range(n_K) for q in range(n_K))
eg1.addConstrs(C[k] >= r[k] + 6 + 1 for k in range(n_K)) # 7 = 6單位的清潔+1單位的提前到點
eg1.addConstrs(C[k] >= 0 for k in range(n_K))
eg1.addConstrs(C[k] <= Cmax for k in range(n_K))


{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>}

In [213]:
eg1.optimize()
print("z* = ", eg1.ObjVal)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 98 rows, 983 columns and 316 nonzeros
Model fingerprint: 0x4921188b
Model has 75 quadratic constraints
Variable types: 0 continuous, 983 integer (954 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [1e+00, 7e+00]
Presolve removed 0 rows and 879 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -


AttributeError: Unable to retrieve attribute 'ObjVal'